- In ORM mode, declarative mapping defines **at once both** a Python object model, as well as database metadata that describes real SQL tables that exist, or will exist.
- The association between Python types and SQL types can be customized using the [type annotation map](https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#customizing-the-type-map).
    - Enums or Litteral can be used as SQL `Enums`

## Attributes of Mapped Classes

In SQLAlchemy, a "column" in the context of a mapped class can be one of the following:
- `mapped_column`: a database column that is mapped to an attribute of the class, typically used for storing raw data types (e.g., integers, strings, dates).
- `relationship`:  a relationship between tables. Used to establish one-to-many, many-to-one, or many-to-many associations between models.
- `hybrid_property`: a computed attribute that behaves like a column, which can be used in both Python code and SQL queries.
- Computed Columns (`Computed`):  a read-only column whose value is derived from a SQL expression, defined at the database level and computed during query execution.
- `association_proxy`: an attribute that **proxies** a relationship, allowing access to related objects in a simplified way. Commonly used in many-to-many relationships or when you want to access attributes of related objects directly.
- `synonym`: maps a class property to an existing column or relationship under a different name.
-  Association Tables: While not technically a column, association tables are used in many-to-many relationships and often act like intermediary columns. Usually defined with the `Table` object.

### `mapped_column` main arguments

| Argument          | Description                                                                  | Usage                                                                                     | Default Value             | Dataclass Mode Only       |
|-------------------|------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|---------------------------|---------------------------|
| `primary_key`     | Indicates if the column is part of the primary key for the table.           | `primary_key=True`. Note that you don't need to explicitly set `nullable=False` or `index=True` for primary keys. | `False`                  | ❌                       |
| `index`           | Adds an index to the column for faster lookups.                             | `index=True`                                                                             | `False`                  | ❌                       |
| `insert_default`  | Specifies a value or callable to be used as a default when inserting rows.   | `insert_default=datetime.utcnow`                                                        | `None`                   | ❌                       |
| `server_default`  | Defines a default value generated or provided by the database server.       | `server_default=func.now()`                                                             | `None`                   | ❌                       |
| `server_onupdate` | Specifies a server-side value to be applied when a row is updated.          | `server_onupdate=func.now()` to update the column to the current time on update.         | `None`                   | ❌                       |
| `unique`          | Specifies whether the column value must be unique across all rows.          | `unique=True`                                                                            | `False`                  | ❌                       |
| `nullable`        | Indicates whether the column can store `NULL` values.                       | `nullable=False` to disallow `NULL` values.                                             | `True`                   | ❌                       |
| `onupdate`        | Specifies a value or callable to apply when the row is updated.             | `onupdate=datetime.utcnow`                                                              | `None`                   | ❌                       |
| `info`            | Provides a place to store custom metadata about the column.                 | `info={"description": "User's email address"}`                                          | `None`                   | ❌                       |
| `autoincrement`   | Controls whether the column's value auto-increments for new rows.           | `autoincrement=True`                                                                     | Database-Dependent        | ❌                       |
| `deferred`        | Normally, when you query a mapped object, SQLAlchemy retrieves all the columns for that object immediately. By marking a column as deferred, the data for that column is excluded from the initial SQL query, and only fetched when you access the attribute for the first time, triggering a separate query to the database. | `deferred=True` to make the column lazily loaded. | `False`                  | ❌                       |
| `comment`         | Adds a database-level comment to the column. This is included in the generated database schema and can be viewed using database tools or SQL queries. | `comment="The user's email address"` | `None`                   | ❌                       |
| `doc`             | Adds a Python-side documentation string to the column. This is purely for the benefit of developers working with the Python code. | `doc="This column stores user email addresses."` | `None`                   | ❌                       |
| `init`            | Determines whether this column is included as a parameter in `__init__`.    | `init=False` to exclude the column from the constructor (usual with pks).                | `True`                   | ✅                       |
| `default`         | Specifies a default value for the column (in Python).                       | `default="active"`                                                                       | `None`                   | ✅                       |
| `default_factory` | Similar to `default`, but uses a callable to generate default values.       | `default_factory=datetime.utcnow`                                                       | `None`                   | ✅                       |


### `relationship` main arguments

| Argument             | Description                                                                                      | Usage                                                                                     | Default Value             |
|----------------------|--------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|---------------------------|
| `back_populates`     | Specifies the name of the complementary property on the related class for bidirectional relationships. Use it when you want explicit control over both sides of the relationship. Use `backref` otherwise. | `back_populates="user"`                                                                   | `None`                   |
| `backref`            | Automatically creates a complementary relationship on the related class. Use it when both sides of the relationship can share the same configuration. Use `back_populates` otherwise.                       | `backref="user"`                                                                          | `None`                   |
| `cascade`            | Defines how operations on a parent object (e.g., delete) cascade to its related objects.         | `cascade="all, delete-orphan"`                                                            | `None`                   |
| `lazy`               | Controls when and how the related objects are loaded.                                            | `lazy="select"` (default), `lazy="joined"`, `lazy="subquery"`, etc.                       | `"select"`               |
| `secondary`          | Defines the intermediary table for many-to-many relationships.                                    | `secondary=association_table`                                                            | `None`                   |
| `order_by`           | Specifies the order in which related objects are loaded.                                         | `order_by=Address.id`                                                                    | `None`                   |
| `viewonly`           | Marks the relationship as read-only, meaning it doesn’t participate in persistence operations (e.g., insert, update, or delete). This is typically used for computed or derived relationships that are not managed by SQLAlchemy. | `viewonly=True`                                                                          | `False`                  |
| `post_update`        | Resolves circular dependencies by issuing a second update statement for related objects.         | `post_update=True`                                                                       | `False`                  |
| `innerjoin`          | Forces the use of an inner join instead of an outer join when loading related objects.           | `innerjoin=True`                                                                         | `False`                  |
| `sync_backref`       | Synchronizes the in-memory state of backrefs automatically.                                      | `sync_backref=False` to disable synchronization.                                         | `True`                   |
| `info`               | Provides a place to store custom metadata about the relationship.                                | `info={"description": "User's addresses"}`                                               | `None`                   |
| `uselist`            | Determines if the relationship should be treated as a collection or a scalar.                    | `uselist=False` for a one-to-one relationship.                                           | `True`                   |
| `passive_deletes`    | Controls whether SQLAlchemy will issue DELETE statements on related objects or rely on database-level ON DELETE behavior. | `passive_deletes=True`                                                                   | `False`                  |
| `foreign_keys`       | Specifies the foreign key(s) used in the relationship, useful in cases of self-referential relationships or multiple foreign keys. | `foreign_keys=[Address.user_id]`                                                         | `Inferred automatically` |



## Class-Level Attributes for SQLAlchemy Models

- `__tablename__`: Defines the name of the table in the database. Must be unique within the schema.
- `__table_args__`: Used to define table-level constraints (e.g., composite keys, indexes) or database-specific options.
- `__mapper_args__`: Configures ORM-specific options like polymorphism, default ordering, and inheritance settings.
- `__abstract__`: Marks a class as abstract, meaning it is not directly mapped to a table but serves as a base class for other models.
- `__init__`: Customizes how instances of the model are initialized.

### `__mapper_args__` Options

| **Option**               | **Description and Usage**                                                                                   | **Example**                                                                                                                |
|--------------------------|------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------|
| **`order_by`**           | Specifies the default ordering of rows for queries on this model. Use this to define a default `ORDER BY` clause. | `__mapper_args__ = {"order_by": name}`                                                                                     |
| **`polymorphic_on`**      | Defines the column used to differentiate between subclasses in a polymorphic inheritance hierarchy.         | `__mapper_args__ = {"polymorphic_on": type}`                                                                               |
| **`polymorphic_identity`**| Assigns a unique identifier to a class in a polymorphic inheritance hierarchy. Use this to map specific subclasses. | `__mapper_args__ = {"polymorphic_identity": "dog"}`                                                                        |
| **`version_id_col`**      | Specifies a column used for optimistic concurrency control. Use this to track and handle version conflicts during updates. | `__mapper_args__ = {"version_id_col": version_id}`                                                                         |
| **`eager_defaults`**      | Forces default values for columns to be loaded from the database after an insert. Use this for server-side default values. | `__mapper_args__ = {"eager_defaults": True}`                                                                               |
| **`confirm_deleted_rows`**| Ensures rows targeted for deletion are checked for existence before being removed. Use this for cases where database constraints are unreliable. | `__mapper_args__ = {"confirm_deleted_rows": False}`                                                                        |
| **`inherit_condition`**   | Defines the condition for joining a subclass table with its parent in a joined-table inheritance setup. Use this to customize inheritance behavior. | `__mapper_args__ = {"inherit_condition": id == Parent.id}`                                                                 |
| **`with_polymorphic`**    | Controls how polymorphic loading is performed. Use this to specify which subclasses should be included in queries. | `__mapper_args__ = {"with_polymorphic": "*"}  # Includes all subclasses`                                                   |


### `__table_args__` Options

| **Option**               | **Description and Usage**                                                                                   | **Example**                                                                                                                |
|--------------------------|------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------|
| **`PrimaryKeyConstraint`** | Defines a composite primary key involving multiple columns. Use this when more than one column forms the primary key. | `PrimaryKeyConstraint("col1", "col2")`                                                                                     |
| **`UniqueConstraint`**    | Ensures uniqueness across one or more columns. Use for multi-column uniqueness constraints.               | `UniqueConstraint("email", "company_id", name="uq_email_company")`                                                         |
| **`Index`**               | Defines a table-level index, which can span multiple columns or use advanced properties. Use for composite indexes or custom options. | `Index("ix_name_age", "name", "age")`                                                                                      |
| **`CheckConstraint`**     | Adds a SQL-level check constraint to enforce conditions on column values. Use for validations like ensuring positive numbers or custom conditions. | `CheckConstraint("price > 0 AND quantity >= 0", name="check_positive_price_quantity")`                                      |
| **`ForeignKeyConstraint`**| Defines multi-column foreign key constraints between tables. Use for relationships involving composite foreign keys. | `ForeignKeyConstraint(["col1", "col2"], ["other_table.col1", "other_table.col2"])`                                          |
| **`schema`**              | Specifies the schema under which the table should be created. Use this when working with multiple schemas in your database. | `{"schema": "my_schema"}`                                                                                                   |                                                            |